In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import joblib

from xgboost import XGBClassifier


In [4]:
df = pd.read_csv("dataset.csv")
df.head()


,id,product_A_sold_in_the_past,product_B_sold_in_the_past,product_A_recommended,product_A,product_C,product_D,cust_hitrate,cust_interactions,cust_contracts,opp_month,opp_old,competitor_Z,competitor_X,competitor_Y,cust_in_iberia,target_variable
0,1,-0.25992,-0.34794,-0.1097,-0.0891,-0.02372,-0.04247,0.12643,0.30731,-0.34997,-1.41464,-0.28185,0,0,0,1,1
1,2,-0.25992,-0.28599,-0.1097,-0.0891,-0.02372,-0.04247,0.42465,3.26349,0.92184,-1.11923,3.54793,1,0,0,0,0
2,3,-0.25992,-0.18998,-0.1097,-0.0891,-0.02372,-0.04247,0.72559,1.29633,-0.34997,0.65323,-0.28185,0,0,0,0,0
3,4,0.65945,3.35306,-0.1097,-0.0891,-0.02372,-0.04247,1.55091,-0.68170,3.46544,1.24404,-0.28185,0,0,0,1,0
4,5,-0.25992,-0.24623,-0.1097,-0.0891,-0.02372,-0.04247,-0.50193,0.65510,-0.03202,1.53945,-0.28185,1,0,0,0,0


In [5]:
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35899 entries, 0 to 35898
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          35899 non-null  int64  
 1   product_A_sold_in_the_past  35899 non-null  float64
 2   product_B_sold_in_the_past  35899 non-null  float64
 3   product_A_recommended       35899 non-null  float64
 4   product_A                   35899 non-null  float64
 5   product_C                   35899 non-null  float64
 6   product_D                   35899 non-null  float64
 7   cust_hitrate                35899 non-null  float64
 8   cust_interactions           35899 non-null  float64
 9   cust_contracts              35899 non-null  float64
 10  opp_month                   35899 non-null  float64
 11  opp_old                     35899 non-null  float64
 12  competitor_Z                35899 non-null  int64  
 13  competitor_X                358

,id,product_A_sold_in_the_past,product_B_sold_in_the_past,product_A_recommended,product_A,product_C,product_D,cust_hitrate,cust_interactions,cust_contracts,opp_month,opp_old,competitor_Z,competitor_X,competitor_Y,cust_in_iberia,target_variable
count,35899.000000,35899.000000,3.589900e+04,35899.000000,3.589900e+04,35899.000000,35899.000000,3.589900e+04,3.589900e+04,35899.000000,3.589900e+04,35899.000000,35899.000000,35899.000000,35899.000000,35899.000000,35899.000000
mean,17950.000000,-0.000004,6.100448e-08,0.000002,-9.822001e-07,-0.000004,0.000001,-3.562773e-07,-2.897017e-08,-0.000001,-1.607287e-07,0.000004,0.268197,0.001978,0.205382,0.603109,0.470487
std,10363.292993,1.000015,1.000014e+00,1.000014,1.000014e+00,1.000014,1.000014,1.000014e+00,1.000014e+00,1.000014,1.000014e+00,1.000013,0.443027,0.044429,0.403986,0.489260,0.499135
min,1.000000,-0.259920,-3.479400e-01,-0.109700,-8.910000e-02,-0.023720,-0.042470,-1.166700e+00,-6.817000e-01,-0.349970,-1.414640e+00,-0.281850,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8975.500000,-0.259920,-3.479400e-01,-0.109700,-8.910000e-02,-0.023720,-0.042470,-9.302400e-01,-6.817000e-01,-0.349970,-8.238200e-01,-0.281850,0.000000,0.000000,0.000000,0.000000,0.000000
50%,17950.000000,-0.259920,-3.479400e-01,-0.109700,-8.910000e-02,-0.023720,-0.042470,-2.187200e-01,-4.317300e-01,-0.349970,-2.330000e-01,-0.281850,0.000000,0.000000,0.000000,1.000000,0.000000
75%,26924.500000,-0.259920,-2.253200e-01,-0.109700,-8.910000e-02,-0.023720,-0.042470,8.437200e-01,3.616600e-01,-0.349970,9.486400e-01,-0.281850,1.000000,0.000000,0.000000,1.000000,1.000000
max,35899.000000,7.953860,8.546760e+00,25.323020,1.878559e+01,65.047500,36.231570,1.730920e+00,1.322972e+01,10.460370,1.834860e+00,3.547930,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
X = df.drop(columns=["target_variable"])
y = df["target_variable"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
model = XGBClassifier(
    n_estimators=200,        # nombre d’arbres
    max_depth=5,             # profunditat màxima dels arbres
    learning_rate=0.1,       # taxa d’aprenentatge
    subsample=0.8,           # fracció de dades per cada arbre
    colsample_bytree=0.8,    # fracció de features per cada arbre
    scale_pos_weight=1,      # ajust per classes desbalancejades (pots modificar-ho)
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"
)

model.fit(X_train_scaled, y_train)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:37:42] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [12]:
y_pred = model.predict(X_test_scaled)


In [13]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))


Accuracy: 0.7300835654596101
F1: 0.7083960276858261
Precision: 0.7203182374541004
Recall: 0.6968620485494376
